# Scaling and vector addition


In [ ]:
%cd drive/My Drive/learning/data_science/coding_the_matrix/resources/

[Errno 2] No such file or directory: 'drive/My Drive/learning/data_science/coding_the_matrix/resources/'
/content


In [ ]:
# Copyright 2013 Philip N. Klein
"""
This file contains a simple plotting interface, which uses an Iframe to
present a plot of points represented as either complex numbers or 2-vectors.
"""
from numbers import Number
from IPython.display import IFrame,HTML, display
import tempfile
import os
import atexit
from io import StringIO


def plot(L, scale=4, dot_size = 3):
    """ plot takes a list of points, optionally a scale (relative to a 200x200 frame),
        optionally a dot size (diameter) in pixels.It produces a string with 
        SVG representing the given plot, and opens the file in an Iframe. It returns nothing.
    """
    scalar = 200./scale
    origin = (210, 210)
    out_str = StringIO()
    out_str.write(
            """<html>
            <head>
            <title>plot</title>
            </head>
            <body>
            <svg height="450" width="450" xmlns="http://www.w3.org/2000/svg">\n'
            <line x1="0" y1="210" x2="420" y2="210"
            style="stroke:rgb(0,0,0);stroke-width:2"/>
            <line x1="210" y1="0" x2="210" y2="420"
            style="stroke:rgb(0,0,0);stroke-width:2"/>""")
    parsedImgPts = list()
    for pt in L:
      if isinstance(pt, Number):
        x,y = pt.real, pt.imag
      else:
          if isinstance(pt, tuple) or isinstance(pt, list):
            x,y = pt
          else:
            raise ValueError
      parsedImgPts.append((origin[0]+scalar*x, origin[1]-scalar*y)) 
    for (a, b) in parsedImgPts:       
        out_str.write('<circle cx="%d" cy="%d" r="%d" fill="red"/>\n'
                          % (a,b , dot_size))
    out_str.write("""</svg>\n</body>\n</html>""")
    display(HTML(out_str.getvalue()))
    out_str.close()
        

In [ ]:
def add(v,w):
  return [v[0] + w[0], v[1] + w[1]]
def addn(v,w):
  return [v[i] + w[i] for i in range(len(v))]  


In [ ]:
L = [(2,2), (3,2), (1.75,1), (2,1), (2.25,1), (2.5,1), (2.75,1), (3,1), (3.25,1)]
plot(L)

In [ ]:
def scalar_vector_mult(alpha, v):
  return [alpha * x for x in v]

In [ ]:
v = [3,2]
plot([scalar_vector_mult(i/10,v) for i in range(11)], 5)

In [ ]:
plot([scalar_vector_mult(i/100,v) for i in range(100)] , 5)

# Dictionary representaion

Class representation of a vector

In [31]:
class Vec:
  def __init__(self, labels, function):
    self.D = labels
    self.f = function

  def __repr__(self) -> str:
    return f"functions {self.f} domains {self.D}"  

  def __add__(u : Vec,v : Vec) -> Vec:
    "Returns the sum of the two vectors"
    assert u.D == v.D
    return Vec(u.D, { x : y for (x, y) in zip(u.f , v.f) } )

 

In [ ]:
v = Vec({'A', 'B' , 'C'}, {'A': 1.})

In [ ]:
for d in v.D:
  if d in v.f:
    print(v.f[d])

1.0


procedure zero_vec(D) with the following spec:
input: set D
output: an instance of Vec representing a D-vector all of whose entries have value zero 

In [ ]:
def zero_vec(D): {d:0 for d in D }

Setter

In [ ]:
def setitems(v, d, val): v.f[d] = val

getter
procedure getitem(v, d) with below spec
input: an instance of v of Vec. and an element d of the set v.D
output: the value of entry d of v

In [ ]:
def getitems(v , d): return v.f[d] if d in v.f else 0

In [32]:
sum2= Vec({'A', 'B' , 'C'}, {'A': 1.}) + Vec({'A', 'B' , 'C'}, {'C': 1.})
print(sum2)

functions {'A': 'C'} domains {'C', 'B', 'A'}


procedure list2vec(L)
input: a list L of field elements
output: an instance of V of Vec with domain {0,1,2...len(L) - 1} such that v[i] = L[i] for each integer i in the domain

In [34]:
def list2vec(L: list)-> Vec:
  return (Vec(set(rangelen((L)))), {k:x for k,x in enumerate(L)})